## Главные задачи, которые должен выполнять скрипт:

- Извлекать реплики с приветствием – где менеджер поздоровался.
- Извлекать реплики, где менеджер представил себя.
- Извлекать имя менеджера.
- Извлекать название компании.
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

### **Рекомендации**:
Сделать локальную копию файла test_data.csv, в исходнике никакие данные не менять!
Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
Для выполнения задачи можно использовать любые библиотеки и NLP модели.
Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей.
Попробуйте использовать другие методы извлечения данных сущностей.

In [107]:
import pandas as pd
import numpy as np
import re
import spacy
import string
from nltk.stem import PorterStemmer
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy.lang.ru.stop_words import STOP_WORDS
import nltk
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from spacy.matcher import Matcher

from nltk.tokenize import word_tokenize, sent_tokenize

In [108]:
#supress warnings
import warnings
warnings.filterwarnings('ignore')

In [109]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)


In [110]:
import ssl # для проверки подключения к интернету

In [111]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

In [112]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words_ru = nltk.corpus.stopwords.words("russian")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maxal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [113]:
stop_words_ru = nltk.corpus.stopwords.words("russian")

In [114]:
# Загрузка данных
data = pd.read_csv('test_data.csv')
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [115]:
# Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.


In [116]:
data['greeting'] = False
data['introduction'] = False
data['manager_name'] = False
data['company_name'] = False
data['goodbye'] = False


In [117]:
data.head()

,dlg_id,line_n,role,text,greeting,introduction,manager_name,company_name,goodbye
0,0,0,client,Алло,False,False,False,False,False
1,0,1,manager,Алло здравствуйте,False,False,False,False,False
2,0,2,client,Добрый день,False,False,False,False,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,False,False,False
4,0,4,client,Ага,False,False,False,False,False


In [118]:
txt = data['text']

In [119]:
#!python -m spacy download ru_core_news_sm

In [120]:
# Загрузка модели для обработки текста

In [121]:
nlp = spacy.load("ru_core_news_sm")


In [122]:
# нормализация данных текстовых полей


In [123]:
#my_stop = {'a'}
stop_words = stop_words_ru
punctuations = string.punctuation

def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    # Создадим токены, которые будем использовать для создания документов с лингвистическими аннотациями
    # грубо говоря мы создаём токены без мусорных слов
    
    mytokens = nlp(sentence)
    porter = PorterStemmer()

    # лематизация
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # удаляем стоп слова и пунктуацию
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    return mytokens

In [124]:
# Применение токенизатора к данным

In [125]:
data['tokens']=data['text'].apply(spacy_tokenizer)

In [126]:
data.head()

,dlg_id,line_n,role,text,greeting,introduction,manager_name,company_name,goodbye,tokens
0,0,0,client,Алло,False,False,False,False,False,[алло]
1,0,1,manager,Алло здравствуйте,False,False,False,False,False,"[алло, здравствовать]"
2,0,2,client,Добрый день,False,False,False,False,False,"[добрый, день]"
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,False,False,False,"[звать, ангелина, компания, диджитал, бизнес, ..."
4,0,4,client,Ага,False,False,False,False,False,[ага]


In [127]:
#соеденим dataframe с текстовыми полями в одну строку
data['text_normalized'] = data['tokens'].apply(lambda x: ' '.join(x))


In [128]:
text_for_world_cloud = sum(data['tokens'].tolist(), [])

In [129]:
text_for_world_cloud

['алло',
 'алло',
 'здравствовать',
 'добрый',
 'день',
 'звать',
 'ангелина',
 'компания',
 'диджитал',
 'бизнес',
 'звоним',
 'повод',
 'продление',
 'лицензия',
 'серый',
 'скоро',
 'срок',
 'заканчиваться',
 'ага',
 'угу',
 'возможный',
 'рассмотреть',
 'другие',
 'вариант',
 'видеть',
 'это',
 'хороший',
 'практика',
 'сравнивать',
 'работать',
 'компания',
 'которая',
 'нам',
 'подливать',
 'поэтому',
 'спасибо',
 'огромный',
 'момент',
 'работать',
 'устраивать',
 'сопровождение',
 'поэтому',
 'угу',
 'обращать',
 'внимание',
 'выбор',
 'выбор',
 'обращать',
 'внимание',
 'дома',
 'выбрать',
 'приоритет',
 'поэтому',
 'тип',
 'параметр',
 'выбор',
 'выбор',
 'история',
 'сотрудничество',
 'которая',
 'важный',
 'смотреть',
 '1',
 'звоните',
 'работать',
 'давно',
 'почему',
 'звоните',
 'течение',
 'свой',
 'время',
 'работать',
 'срок',
 'заканчиваться',
 'поэтому',
 'набрать',
 'ко',
 'всему',
 'совершенно',
 'верный',
 'делать',
 '50',
 'компания',
 'которые',
 'звонить',
 'с

In [130]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [131]:
# data['name'] = data['text_normalized'].apply(lambda x: names_extractor.extract_name(x))

In [132]:
data['names_extracted']=data['text_normalized'].apply(names_extractor.find)

In [133]:
new_cols_df = data['names_extracted'].apply(lambda x: x.fact.as_json if x else None).apply(pd.Series)

In [134]:
new_cols_df

,last,first
0,NaN,NaN
1,здравствовать,NaN
2,добрый,NaN
3,NaN,ангелина
4,NaN,NaN
...,...,...
475,повод,NaN
476,мочь,NaN
477,спасибо,NaN
478,спасибо,NaN


In [135]:
#concat new_cols_df to dataframe
data = pd.concat([data, new_cols_df], axis=1)
data

,dlg_id,line_n,role,text,greeting,introduction,manager_name,company_name,goodbye,tokens,text_normalized,names_extracted,last,first
0,0,0,client,Алло,False,False,False,False,False,[алло],алло,None,NaN,NaN
1,0,1,manager,Алло здравствуйте,False,False,False,False,False,"[алло, здравствовать]",алло здравствовать,"Match(start=5, stop=18, fact=Name(first=None, ...",здравствовать,NaN
2,0,2,client,Добрый день,False,False,False,False,False,"[добрый, день]",добрый день,"Match(start=0, stop=6, fact=Name(first=None, l...",добрый,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,False,False,False,"[звать, ангелина, компания, диджитал, бизнес, ...",звать ангелина компания диджитал бизнес звоним...,"Match(start=6, stop=14, fact=Name(first='ангел...",NaN,ангелина
4,0,4,client,Ага,False,False,False,False,False,[ага],ага,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,False,False,False,"[повод, виджет, ними, обсудить, конкретно, про...",повод виджет ними обсудить конкретно продам,"Match(start=0, stop=5, fact=Name(first=None, l...",повод,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,False,False,False,"[весь, высылать, счёт, вами, связь, будут, воп...",весь высылать счёт вами связь будут вопрос моч...,"Match(start=43, stop=47, fact=Name(first=None,...",мочь,NaN
477,5,140,client,Спасибо спасибо,False,False,False,False,False,"[спасибо, спасибо]",спасибо спасибо,"Match(start=0, stop=7, fact=Name(first=None, l...",спасибо,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,False,False,False,"[созвонимся, ага, спасибо, давать]",созвонимся ага спасибо давать,"Match(start=15, stop=22, fact=Name(first=None,...",спасибо,NaN


In [136]:
# Поиск и обработка реплик приветствия

nlp = spacy.load('ru_core_news_sm', disable=["ner", "parser"])
ruler = nlp.add_pipe("entity_ruler") #назначение именнованных правил шаблонов
matcher = Matcher(nlp.vocab) #поисковик по словарю модели spacy

def add_entity(unique_ds, label):

    #Create the EntityRuler
    for ent in unique_ds:
        ruler.add_patterns([{"label": label, "pattern":ent}])
        # ruler.add_patterns([{"label": "greeting", "pattern":a}])
    return ruler

In [137]:
unique_gt =["Добрый день", "Здравствуйте", "здравствуйте", "Алло"]
ruler = add_entity(unique_gt, "greeting")

In [138]:
pattern1 =[[{'ENT_TYPE': 'greeting'}]]
matcher.add("gret1",pattern1)

In [139]:
pattern2 =[[{'ENT_TYPE': 'greeting'},{'ENT_TYPE': 'greeting'}]]
matcher.add("gret2",pattern2)

In [140]:
#names pattern

In [141]:
unique_names = list(data['first'].unique())[1:]

In [142]:
unique_names

['ангелина',
 'хирам',
 'поле',
 'сапаров',
 'астана',
 'любой',
 'дмитрий',
 'айдар',
 'галочка',
 'анастасия',
 'амо',
 'вячеслав',
 'максим']

In [143]:
# unique_names
ruler = add_entity(unique_names, "name")

In [144]:
pattern3 =[[{'ENT_TYPE': 'name'}]]
matcher.add("name",pattern3)

In [145]:
#company pattern

business_names = ["диджитал бизнес"]
ruler = add_entity(business_names, "company")

In [146]:
pattern4 =[[{'ENT_TYPE': 'company'}]]
matcher.add("company",pattern4)

In [147]:
#goodbye pattern
goodbye = ["пока", "до свидания", "до встречи"]
ruler = add_entity(goodbye, "goodbye")

In [148]:
pattern5 =[[{'ENT_TYPE': 'goodbye'}]]
matcher.add("goodbye",pattern5)


In [149]:
pattern5_1 =[[{'ENT_TYPE': 'goodbye'}, {'ENT_TYPE': 'goodbye'}]]
matcher.add("goodbye1",pattern5_1)

In [150]:
def matches_in_df(matcher):
    match_list = []
    text_list = []
    ind_list = []

    for i in range(len(data["text"])):
        
        
        try:
            
            text = data["text"].iloc[i]
            doc = nlp(text)

            matches = matcher(doc)
            for match_id, start, end in matches:
                
                string_id = nlp.vocab.strings[match_id]  # Get string representation
                span = doc[start:end]  # The matched span
          #     print(match_id, string_id, start, end, span.text)
          #     print(string_id, span.text, i)
                match_list.append(string_id)
                text_list.append(span.text)
                ind_list.append(i)
          #     print(i)
        
        except:
            
            match_list.append("null")
            text_list.append("null")
            ind_list.append(i)
            pass

    df_matches = pd.DataFrame(columns =['match', 'text', 'id'])
    df_matches['match']=match_list
    df_matches['text']=text_list
    df_matches['id']=ind_list

    return df_matches



In [151]:
matches_in_df(matcher)

,match,text,id
0,gret1,Алло,0
1,gret1,Алло,1
2,gret2,Алло здравствуйте,1
3,gret1,здравствуйте,1
4,gret1,Добрый,2
...,...,...,...
63,gret1,Алло,455
64,name,дмитрий,472
65,goodbye,до,479
66,goodbye1,до свидания,479


In [152]:
index_greet = matches_in_df(matcher)[(matches_in_df(matcher)['match']=='gret1') | (matches_in_df(matcher)['match']=='gret2') ]['text']

In [153]:
index_name = matches_in_df(matcher)[(matches_in_df(matcher)['match']=='name')]['text']

In [154]:
index_company = matches_in_df(matcher)[(matches_in_df(matcher)['match']=='company')]['text']

In [155]:
list(index_greet.unique())


['Алло',
 'Алло здравствуйте',
 'здравствуйте',
 'Добрый',
 'Добрый день',
 'день',
 'Здравствуйте']

In [156]:
list(index_name.unique())

['ангелина',
 'астана',
 'дмитрий',
 'максим',
 'анастасия',
 'амо',
 'айдар',
 'любой',
 'вячеслав']

In [157]:
list(index_company.unique())

['диджитал', 'бизнес']

In [158]:
index_goodbye = matches_in_df(matcher)[(matches_in_df(matcher)['match']=='goodbye') | (matches_in_df(matcher)['match']=='goodbye1') ]['text']

In [159]:
list(index_goodbye.unique())

['до', 'до свидания', 'свидания', 'пока']

In [160]:
#Извлекать реплики с приветствием – где менеджер поздоровался.
for i in range(len(txt)):
    if any(txt[i].find(x) != -1 for x in list(index_greet.unique())):
        data['greeting'][i] = 1
    else:
        data['greeting'][i] = 0


In [161]:
#проверка строк где поздровались
data[data['greeting']==1]


,dlg_id,line_n,role,text,greeting,introduction,manager_name,company_name,goodbye,tokens,text_normalized,names_extracted,last,first
0,0,0,client,Алло,True,False,False,False,False,[алло],алло,None,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,False,False,False,False,"[алло, здравствовать]",алло здравствовать,"Match(start=5, stop=18, fact=Name(first=None, ...",здравствовать,NaN
2,0,2,client,Добрый день,True,False,False,False,False,"[добрый, день]",добрый день,"Match(start=0, stop=6, fact=Name(first=None, l...",добрый,NaN
80,0,80,client,Алло,True,False,False,False,False,[алло],алло,None,NaN,NaN
109,1,0,client,Да здравствуйте когда заканчивается,True,False,False,False,False,"[здравствовать, заканчиваться]",здравствовать заканчиваться,"Match(start=0, stop=13, fact=Name(first=None, ...",здравствовать,NaN
110,1,1,manager,Алло здравствуйте,True,False,False,False,False,"[алло, здравствовать]",алло здравствовать,"Match(start=5, stop=18, fact=Name(first=None, ...",здравствовать,NaN
131,1,22,client,Алло,True,False,False,False,False,[алло],алло,None,NaN,NaN
164,2,0,client,Алло,True,False,False,False,False,[алло],алло,None,NaN,NaN
165,2,1,client,Здравствуйте,True,False,False,False,False,[здравствовать],здравствовать,"Match(start=0, stop=13, fact=Name(first=None, ...",здравствовать,NaN
166,2,2,manager,Алло здравствуйте,True,False,False,False,False,"[алло, здравствовать]",алло здравствовать,"Match(start=5, stop=18, fact=Name(first=None, ...",здравствовать,NaN


In [162]:
# Извлекать реплики с приветствием – где менеджер поздоровался.

In [163]:
# Извлекать реплики, где менеджер представил себя.


In [164]:
for i in range(len(txt)):
    if any(txt[i].find(x) != -1 for x in list(index_name.unique())):
        data['manager_name'][i] = 1
    else:
        data['manager_name'][i] = 0

In [165]:
#проверка строк где менеджер представился
data[data['manager_name']==1]['first']


3       ангелина
26           NaN
111     ангелина
159          NaN
167     ангелина
229       астана
250      дмитрий
251          NaN
253      дмитрий
282          NaN
285          NaN
319        айдар
338    анастасия
346          NaN
358          NaN
364          NaN
375          NaN
410          амо
438     вячеслав
444       максим
448        любой
472      дмитрий
Name: first, dtype: object

In [166]:
data

,dlg_id,line_n,role,text,greeting,introduction,manager_name,company_name,goodbye,tokens,text_normalized,names_extracted,last,first
0,0,0,client,Алло,True,False,False,False,False,[алло],алло,None,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,False,False,False,False,"[алло, здравствовать]",алло здравствовать,"Match(start=5, stop=18, fact=Name(first=None, ...",здравствовать,NaN
2,0,2,client,Добрый день,True,False,False,False,False,"[добрый, день]",добрый день,"Match(start=0, stop=6, fact=Name(first=None, l...",добрый,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,True,False,False,"[звать, ангелина, компания, диджитал, бизнес, ...",звать ангелина компания диджитал бизнес звоним...,"Match(start=6, stop=14, fact=Name(first='ангел...",NaN,ангелина
4,0,4,client,Ага,False,False,False,False,False,[ага],ага,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,False,False,False,"[повод, виджет, ними, обсудить, конкретно, про...",повод виджет ними обсудить конкретно продам,"Match(start=0, stop=5, fact=Name(first=None, l...",повод,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,False,False,False,"[весь, высылать, счёт, вами, связь, будут, воп...",весь высылать счёт вами связь будут вопрос моч...,"Match(start=43, stop=47, fact=Name(first=None,...",мочь,NaN
477,5,140,client,Спасибо спасибо,False,False,False,False,False,"[спасибо, спасибо]",спасибо спасибо,"Match(start=0, stop=7, fact=Name(first=None, l...",спасибо,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,False,False,False,"[созвонимся, ага, спасибо, давать]",созвонимся ага спасибо давать,"Match(start=15, stop=22, fact=Name(first=None,...",спасибо,NaN


In [167]:
#Извлекать реплики с company – где менеджер представил компанию.
for i in range(len(txt)):
    if any(txt[i].find(x) != -1 for x in list(index_company.unique())):
        data['company_name'][i] = 1
    else:
        data['company_name'][i] = 0



In [168]:
#проверка строк где менеджер представил компанию
data[data['company_name']==1]['text']


3      Меня зовут ангелина компания диджитал бизнес з...
111    Меня зовут ангелина компания диджитал бизнес з...
167    Меня зовут ангелина компания диджитал бизнес з...
177                                      Диджитал бизнес
178                                Здесь же это у бизнес
184    Вот и смотрите на счет кэшбэка то что у вас кэ...
235    Не перепутала оставлю тогда на ватсап хорошо а...
251    Добрый меня максим зовут компания китобизнес у...
268    Так а вот подскажите пожалуйста может быть про...
282    Да нет конечно не сложно да ну смотрите да мы ...
285    А самого внедрения процесса ну я я бы вам реко...
373    Конкретно под ваш бизнес разговаривать и вам у...
421    А там раз в неделю будет как то диагностироват...
444    Вот и вам с вами свяжется максим он отвечает у...
447    Вот и после этого он уже вам даст рекомендации...
448    То есть это уже будете обсуждать с ним и если ...
Name: text, dtype: object

In [169]:
#Извлекать реплики с goodbye – где менеджер прощался.
for i in range(len(txt)):
    if any(txt[i].find(x) != -1 for x in list(index_goodbye.unique())):
        data['goodbye'][i] = 1
    else:
        data['goodbye'][i] = 0


In [170]:
#проверка строк где менеджер прощался
data[data['goodbye']==1]['text']


7      Как как бы уже до этого момента работаем все у...
10     Да на выбор я уже не обращаю внимание я уже до...
23     Незначительная но бесплатная и покажите свою к...
26     Ну лояльности спрашивается 2 месяц не показыва...
30     А как будет недостаточно хорошие качественные ...
35     Слушайте очень сложно сказать потому что видже...
59     По какой причине я должен принять решение в ва...
68     Ну не факт что вы не можете получить допустим ...
84                                        Решать договор
95     Ну просто подумайте подумайте вашей командой к...
96     Потому что это ну вот эти звонки за месяц до о...
97     Мне конечно интересно какая у вас конверсия та...
108                           Всего хорошего до свидания
123                     Мне пока больше не надо наверное
125    65 а нет не там еще 1 2 3 4 5 нет правильно ну...
129    564 р тогда такой вопрос и это допустим сейчас...
133    Сейчас прямо здесь скажите пожалуйста в пользо...
163                            

In [171]:
# создать столбец с названием culture, в котором 1 если в строке greeting 1 и в строке goodbye 1, иначе 0
data['culture'] = data['greeting'] + data['goodbye']
data['culture'] = data['culture'].apply(lambda x: 1 if x == 2 else 0)


In [172]:
data


,dlg_id,line_n,role,text,greeting,introduction,manager_name,company_name,goodbye,tokens,text_normalized,names_extracted,last,first,culture
0,0,0,client,Алло,True,False,False,False,False,[алло],алло,None,NaN,NaN,0
1,0,1,manager,Алло здравствуйте,True,False,False,False,False,"[алло, здравствовать]",алло здравствовать,"Match(start=5, stop=18, fact=Name(first=None, ...",здравствовать,NaN,0
2,0,2,client,Добрый день,True,False,False,False,False,"[добрый, день]",добрый день,"Match(start=0, stop=6, fact=Name(first=None, l...",добрый,NaN,0
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,True,True,False,"[звать, ангелина, компания, диджитал, бизнес, ...",звать ангелина компания диджитал бизнес звоним...,"Match(start=6, stop=14, fact=Name(first='ангел...",NaN,ангелина,0
4,0,4,client,Ага,False,False,False,False,False,[ага],ага,None,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False,False,False,False,"[повод, виджет, ними, обсудить, конкретно, про...",повод виджет ними обсудить конкретно продам,"Match(start=0, stop=5, fact=Name(first=None, l...",повод,NaN,0
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False,False,False,False,"[весь, высылать, счёт, вами, связь, будут, воп...",весь высылать счёт вами связь будут вопрос моч...,"Match(start=43, stop=47, fact=Name(first=None,...",мочь,NaN,0
477,5,140,client,Спасибо спасибо,False,False,False,False,False,"[спасибо, спасибо]",спасибо спасибо,"Match(start=0, stop=7, fact=Name(first=None, l...",спасибо,NaN,0
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,False,False,False,False,False,"[созвонимся, ага, спасибо, давать]",созвонимся ага спасибо давать,"Match(start=15, stop=22, fact=Name(first=None,...",спасибо,NaN,0


In [174]:
"git init

SyntaxError: invalid syntax (<ipython-input-174-f188d5ac1197>, line 1)

In [173]:
git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/Subzero322/test.git
git push -u origin main

SyntaxError: invalid syntax (<ipython-input-173-a0bc2b1db054>, line 1)